### Second Year Project
## Natural Language Processing
Group X -  Fillip Due, Andreas Olsen, Louis Brandt, Emma Bisgaard



### Project Description

Describe siUU

### Project Phase 1: Baseline

In [ ]:
# 12 coming s00n Lets Gooooo